In [ ]:
%run setup.py build_ext --inplace

In [ ]:
from typing import NamedTuple

folder_path = "./pickles/"

class KnotInput(NamedTuple):
    braid: list
    n: int  # number of punctures
    k: int  # number of E-branes

In [ ]:
knots_braids = {
    "3_1_reduced": [1, 2, 1, 2],
    "8_2_reduced": [1, 1, 1, 1, 1, -2, 1, -2],
    "8_5_reduced": [1, 1, 1, -2, 1, 1, 1, -2],
    "8_7_reduced": [-1, -1, -1, -1, 2, -1, 2, 2],
    "8_9_reduced": [1, 1, 1, -2, 1, -2, -2, -2],
    "8_10_reduced": [-1, -1, -1, 2, -1, -1, 2, 2],
    "8_16_reduced": [-1, -1, 2, -1, -1, 2, -1, 2],
    "8_17_reduced": [1, 1, -2, 1, -2, 1, -2, -2],
    "8_18_reduced": [1, -2, 1, -2, 1, -2, 1, -2],
    "8_19_reduced": [1, 1, 1, 2, 1, 1, 1, 2],
    "8_20_reduced": [1, 1, 1, -2, -1, -1, -1, -2],
    "8_21_reduced": [1, 1, 1, 2, -1, -1, 2, 2],
}

In [ ]:
inputs = {}
for name, short_braid in knots_braids.items():
    inputs[name] = KnotInput(
        braid=[2] + [sign(s)*(abs(s)+2) for s in knots_braids[name]] + [-2],
        n=6,
        k=2,
    )

# just to illustrate how to change input
#inputs["8_2"] = inputs["8_2"]._replace(max_dots=1, max_order=3)

inputs

In [ ]:
from klrw.combinatorial_ebranes import CombinatorialEBrane

for name, input_data in inputs.items():
    print("==============================================================")
    print("                        Solving "+name+"                        ")
    print("==============================================================")
    E = CombinatorialEBrane(input_data.n,input_data.k)
    E.apply_braid(input_data.braid)
    try:
        E.link_homology(
            ZZ,
            pickle="save",
            link_name=name,
            folder_path="./",
        )

    except AssertionError:
        print(name+" failed.")

